In [2]:
import cobra
import os
from cobra import Metabolite, Reaction
import escher
from escher import Builder


joiner = lambda e1, e2: os.path.join(e1, e2)
path = '20Z'

## Prepare paths and downloading model

In [ ]:
maker([path])

In [31]:
model = cobra.io.read_sbml_model(joiner(path, 'fbc_iIA409_20Z.xml'))

(None,
 {'SBML_FATAL': [],
  'SBML_ERROR': ['E0 (Error): XML content (core, L1); Missing XML declaration; Missing XML declaration at beginning of XML input.\n',
   'E1 (Error): General SBML conformance (core, L29); Document does not conform to the SBML XML schema; An SBML XML document must conform to the XML Schema for the corresponding SBML Level, Version and Release. The XML Schema for SBML defines the basic SBML object structure, the data types used by those objects, and the order in which the objects may appear in an SBML document.\n'],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': [],
  'COBRA_FATAL': [],
  'COBRA_ERROR': ['No SBML model detected in file.'],
  'COBRA_WARNING': [],
  'COBRA_CHECK': []})

In [104]:
model.compartments

{'e': '', 'p': '', 'c': ''}

## Tungstate adding

In [105]:
w_e = Metabolite(
    'tungs_e',
    name='Tungstate',
    formula='O4W1',
    charge=-2,
    compartment='e'
)
model.add_metabolites([w_e])

In [106]:
model.add_boundary(w_e, type="exchange", lb = 0, ub = 1000)

Reaction identifier,EX_tungs_e
Name,Tungstate exchange
Memory address,0x7fb6c05162b0
Stoichiometry,tungs_e --> Tungstate -->
GPR,
Lower bound,0
Upper bound,1000


In [107]:
w_p = Metabolite(
    'tungs_p',
    name='Tungstate',
    formula='O4W1',
    charge=-2,
    compartment='p'
)


In [108]:
w_c = Metabolite(
    'tungs_c',
    name='Tungstate',
    formula='O4W1',
    charge=-2,
    compartment='c'
)

In [109]:
atp = model.metabolites.atp_c
adp = model.metabolites.adp_c
h = model.metabolites.h_c
h2o = model.metabolites.h2o_c
pi = model.metabolites.pi_c

In [110]:
w_p_transpot = Reaction('TUNGSabcp')
w_p_transpot.name = 'Tungstate transport via ABC system (exchange to periplasm)'
w_p_transpot.subsystem = 'Transport'
w_p_transpot.lower_bound = 0.
w_p_transpot.upper_bound = 1000.

In [111]:
w_p_transpot.add_metabolites({atp: -1, h2o: -1, w_e: -1, adp: 1, h: 1, pi: 1, w_p: 1})

In [112]:
w_transpot = Reaction('TUNGSt')
w_transpot.name = 'Tungstate transport via diffusion (periplasm to cell)'
w_transpot.subsystem = 'Transport'
w_transpot.lower_bound = -0.
w_transpot.upper_bound = 0.

In [113]:
w_transpot.add_metabolites({w_p: -1, w_c: 1})

In [114]:
model.add_reactions([w_p_transpot, w_transpot])

## Lanthanum adding

In [115]:
la_e = Metabolite(
    'la_e',
    name='Lanthanum+3',
    formula='La',
    charge=+3,
    compartment='e'
)
model.add_metabolites([la_e])

In [116]:
model.add_boundary(la_e, type="exchange", lb = 0, ub = 1000)

Reaction identifier,EX_la_e
Name,Lanthanum+3 exchange
Memory address,0x7fb6c0a5ce20
Stoichiometry,la_e --> Lanthanum+3 -->
GPR,
Lower bound,0
Upper bound,1000


In [117]:
la_p = Metabolite(
    'la_p',
    name='Lanthanum+3',
    formula='La',
    charge=+3,
    compartment='p'
)

In [118]:
la_c = Metabolite(
    'la_c',
    name='Lanthanum+3',
    formula='La',
    charge=+3,
    compartment='c'
)

In [119]:
la_p_transpot = Reaction('LAabcp')
la_p_transpot.name = 'Lanthanum transport via ABC system (exchange to periplasm)'
la_p_transpot.subsystem = 'Transport'
la_p_transpot.lower_bound = 0.
la_p_transpot.upper_bound = 1000.

In [120]:
la_p_transpot.add_metabolites({atp: -1, h2o: -1, la_e: -1, adp: 1, h: 1, pi: 1, la_p: 1})

In [121]:
la_transpot = Reaction('LAt')
la_transpot.name = 'Lanthanum transport via diffusion (periplasm to cell)'
la_transpot.subsystem = 'Transport'
la_transpot.lower_bound = -0.
la_transpot.upper_bound = 0.

In [122]:
la_transpot.add_metabolites({la_p: -1, la_c: 1})

In [123]:
model.add_reactions([la_p_transpot, la_transpot])

## Ca reactions modification

In [124]:
ca2_t = model.reactions.Ca2tec

In [125]:
ca_p = Metabolite(
    'ca2_p',
    name='Ca+2',
    formula='Ca',
    charge=+2,
    compartment='p'
)

In [126]:
ca_p_transpot = Reaction('Ca2tep')
ca_p_transpot.name = 'Ca2  transport via ABC system (exchange to periplasm)'
ca_p_transpot.subsystem = 'Transport'
ca_p_transpot.lower_bound = 0.
ca_p_transpot.upper_bound = 1000.
ca_p_transpot.gene_reaction_rule = ca2_t.gene_reaction_rule

In [127]:
ca_p_transpot.add_metabolites({atp: -1, h2o: -1, model.metabolites.ca2_e: -1, adp: 1, h: 1, pi: 1, ca_p: 1})

In [128]:
ca_transpot = Reaction('Ca2t')
ca_transpot.name = 'Ca2 transport via diffusion (periplasm to cell)'
ca_transpot.subsystem = 'Transport'
ca_transpot.lower_bound = -1000.
ca_transpot.upper_bound = 0.
ca_transpot.gene_reaction_rule = ca2_t.gene_reaction_rule

In [129]:
ca_transpot.add_metabolites({ca_p: -1,  model.metabolites.ca2_c: 1})

In [130]:
model.add_reactions([ca_p_transpot, ca_transpot])
model.reactions.Ca2tec.delete()

In [131]:
model.reactions.Ca2tep

Reaction identifier,Ca2tep
Name,Ca2 transport via ABC system (exchange to periplasm)
Memory address,0x7fb6c0590d60
Stoichiometry,atp_c + ca2_e + h2o_c --> adp_c + ca2_p + h_c + pi_c ATP + Ca+2 + H2O --> ADP + Ca+2 + H+ + phosphate
GPR,MEALZ_2422 or MEALZ_2573
Lower bound,0.0
Upper bound,1000.0


## Modification of reactions

### Making FDHTungs

In [132]:
fdh = model.reactions.FDH

In [133]:
fdh.metabolites

{<Metabolite nad_c at 0x7fb6c0764310>: -1.0,
 <Metabolite for_c at 0x7fb6c0764820>: -1.0,
 <Metabolite nadh_c at 0x7fb6c0764340>: 1.0,
 <Metabolite co2_c at 0x7fb6c0764250>: 1.0}

In [134]:
fdh_w = Reaction('FDHTungs')
fdh_w.name = 'Tungsten dependent formate dehydrogenase'
fdh_w.lower_bound = 0.
fdh_w.upper_bound = 1000.
fdh_w.gene_reaction_rule = fdh.gene_reaction_rule

In [135]:
fdh_w.add_metabolites(fdh.metabolites)

In [136]:
fdh_w.add_metabolites({w_p: -1, w_c: 1 })

In [137]:
model.add_reactions([fdh_w])

In [138]:
model.reactions.FDHTungs

Reaction identifier,FDHTungs
Name,Tungsten dependent formate dehydrogenase
Memory address,0x7fb6c059a400
Stoichiometry,for_c + nad_c + tungs_p --> co2_c + nadh_c + tungs_c formate + NAD+ + Tungstate --> CO2 + NADH + Tungstate
GPR,(MEALZ_0215 and MEALZ_0216 and MEALZ_0217) or (MEALZ_1882 and MEALZ_1883)
Lower bound,0.0
Upper bound,1000.0


In [139]:
model.reactions.FDH

Reaction identifier,FDH
Name,formate dehydrogenase
Memory address,0x7fb6c0640cd0
Stoichiometry,for_c + nad_c --> co2_c + nadh_c formate + NAD+ --> CO2 + NADH
GPR,(MEALZ_0215 and MEALZ_0216 and MEALZ_0217) or (MEALZ_1882 and MEALZ_1883)
Lower bound,0.0
Upper bound,1000.0


### Making MXA with Ca

In [140]:
model.reactions.MXA

Reaction identifier,MXA
Name,methanol dehydrogenase (cytochrome cL-mxaF)
Memory address,0x7fb6c0a72ac0
Stoichiometry,cytCLox_p + meoh_p --> cytCLred_p + fald_p + 2.0 h_c cytochrome cL + methanol --> Reduced cytochrome cL + formaldehyde + 2.0 H+
GPR,MEALZ_3445 and MEALZ_3446 and MEALZ_3447 and MEALZ_3448
Lower bound,0.0
Upper bound,1000.0


In [141]:
model.reactions.MXA.add_metabolites({ca_p: -1, model.metabolites.ca2_c: 1 })

In [142]:
model.reactions.MXA

Reaction identifier,MXA
Name,methanol dehydrogenase (cytochrome cL-mxaF)
Memory address,0x7fb6c0a72ac0
Stoichiometry,ca2_p + cytCLox_p + meoh_p --> ca2_c + cytCLred_p + fald_p + 2.0 h_c Ca+2 + cytochrome cL + methanol --> Ca+2 + Reduced cytochrome cL + formaldehyde + 2.0 H+
GPR,MEALZ_3445 and MEALZ_3446 and MEALZ_3447 and MEALZ_3448
Lower bound,0.0
Upper bound,1000.0


### Making MXALa with La and adding MXALa fald -> for

In [143]:
mxala = model.reactions.MXALa

In [144]:
mxala

Reaction identifier,MXALa
Name,methanol dehydrogenase (xoxF)
Memory address,0x7fb6c0a7b7f0
Stoichiometry,cytCLox_p + fald_p --> cytCLred_p + for_p + 2.0 h_c cytochrome cL + formaldehyde --> Reduced cytochrome cL + for[p] + 2.0 H+
GPR,MEALZ_3497
Lower bound,0.0
Upper bound,1000.0


In [145]:
new_mxala = Reaction('MXALa')
new_mxala.name = mxala.name
new_mxala.lower_bound = 0.
new_mxala.upper_bound = 1000.
new_mxala.gene_reaction_rule = mxala.gene_reaction_rule

In [ ]:
mxala_2 = Reaction('MXALa_for')
mxala_2.name = mxala.name
mxala_2.lower_bound = 0.
mxala_2.upper_bound = 0.
mxala_2.gene_reaction_rule = mxala.gene_reaction_rule

In [ ]:
mxala_2.add_metabolites(mxala.metabolites)

In [146]:
mxla_met = mxala.metabolites
del mxla_met[model.metabolites.fald_p]
del mxla_met[model.metabolites.for_p]

In [147]:
new_mxala.add_metabolites({la_p: -1, la_c: 1, model.metabolites.meoh_p: -1, model.metabolites.fald_p: 1})
new_mxala.add_metabolites(mxla_met)

In [148]:
new_mxala

Reaction identifier,MXALa
Name,methanol dehydrogenase (xoxF)
Memory address,0x7fb6c04ede20
Stoichiometry,cytCLox_p + la_p + meoh_p --> cytCLred_p + fald_p + 2.0 h_c + la_c cytochrome cL + Lanthanum+3 + methanol --> Reduced cytochrome cL + formaldehyde + 2.0 H+ + Lanthanum+3
GPR,MEALZ_3497
Lower bound,0.0
Upper bound,1000.0


In [149]:
model.reactions.MXALa.delete()

In [150]:
model.add_reactions([new_mxala])

In [151]:
model.reactions.MXALa

Reaction identifier,MXALa
Name,methanol dehydrogenase (xoxF)
Memory address,0x7fb6c04ede20
Stoichiometry,cytCLox_p + la_p + meoh_p --> cytCLred_p + fald_p + 2.0 h_c + la_c cytochrome cL + Lanthanum+3 + methanol --> Reduced cytochrome cL + formaldehyde + 2.0 H+ + Lanthanum+3
GPR,MEALZ_3497
Lower bound,0.0
Upper bound,1000.0


In [152]:
cobra.io.write_sbml_model(model, joiner(path, 'W_La_Ca_iIA409_20Z.xml'))

In [153]:
cobra.io.save_json_model(model, joiner(path, 'W_La_Ca_iIA409_20Z.json'))

In [3]:
builder_wt = Builder(map_json='20Z/new_20Zmap.json')
#builder_wt.model = m
builder_wt.model = cobra.io.load_json_model(joiner(path, 'W_La_Ca_iIA409_20Z.json'))
solution_wt = builder_wt.model.optimize()
builder_wt.reaction_data = solution_wt.fluxes

In [4]:
builder_wt

Builder(reaction_data={'EX_ch4_e': -11.7, 'EX_o2_e': -13.771798687419345, 'EX_meoh_e': 0.0, 'MEohtep': 0.0, 'E…